In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table 
from scipy.optimize import minimize

In [3]:
from WindPy import w
w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒  
w.isconnected() # 判断WindPy是否已经登录成功

ModuleNotFoundError: No module named 'WindPy'

In [3]:

jc_index_code_list = ["399372.SZ","399373.SZ","399374.SZ","399375.SZ","399376.SZ","399377.SZ"]

sw_index_code_list = ["801811.SI","801812.SI","801813.SI","801821.SI","801822.SI","801823.SI","801831.SI","801832.SI","801833.SI",
                 "801841.SI","801842.SI","801843.SI","801851.SI","801852.SI","801853.SI","801863.SI"]

wind_index_code_list = ["8888701.WI","8888702.WI","8888703.WI","8888704.WI","8888705.WI","8888706.WI",
                        "8888707.WI","8888708.WI","8888709.WI","8888710.WI"]

zxbp_index_code_list = ["816000G.CI","816000PG.CI","816000PV.CI","816000V.CI","CSPSADRP.CI"]

debt_index_code = ['H11001.CSI']

fund_code = ["161005.OF"]

benchmark_code =["000300.SH"]



In [4]:
start_date = "2015-01-01"
end_date = "2020-12-31"

In [5]:
jc_index = w.wsd(jc_index_code_list, "close", start_date, end_date, "PriceAdj=F")
sw_index = w.wsd(sw_index_code_list, "close", start_date, end_date, "PriceAdj=F")
wind_index = w.wsd(wind_index_code_list, "close", start_date, end_date, "PriceAdj=F")
zxbp_index = w.wsd(zxbp_index_code_list, "close", start_date, end_date, "PriceAdj=F")

debt_index = w.wsd(debt_index_code ,"close", start_date, end_date, "PriceAdj=F")

In [6]:
fund = w.wsd(fund_code, "close", start_date, end_date, "PriceAdj=F")             #usedf=True
benchmark = w.wsd(benchmark_code, "close", start_date, end_date, "PriceAdj=F")

In [8]:
#原始价格数据
raw_price = pd.DataFrame({fund_code[0]:fund.Data[0],
                         benchmark_code[0]:benchmark.Data[0],
                         jc_index_code_list[0]:jc_index.Data[0],
                         jc_index_code_list[1]:jc_index.Data[1],
                         jc_index_code_list[2]:jc_index.Data[2],
                         jc_index_code_list[3]:jc_index.Data[3],
                         jc_index_code_list[4]:jc_index.Data[4],
                         jc_index_code_list[5]:jc_index.Data[5],
                         debt_index_code[0]: debt_index.Data[0]},index=fund.Times)


In [9]:
#计算单日收益率
price_return = pd.DataFrame(np.array(raw_price.iloc[:-1,:])/np.array(raw_price.iloc[1:,:])-1,
                            columns= fund_code+benchmark_code+jc_index_code_list+debt_index_code, index=fund.Times[1:])
price_return

,161005.OF,000300.SH,399372.SZ,399373.SZ,399374.SZ,399375.SZ,399376.SZ,399377.SZ,H11001.CSI
2015-01-06,-0.002801,0.000132,0.002043,0.008797,-0.016383,-0.000638,-0.015493,0.001241,-0.000742
2015-01-07,0.002246,-0.000749,0.009454,0.000662,0.002697,-0.004365,0.001455,-0.001157,0.000078
2015-01-08,-0.002241,0.023750,0.022595,0.032590,0.004331,0.017915,0.000869,0.016725,-0.000591
2015-01-09,-0.008333,0.003535,0.001015,-0.002948,0.004996,0.013965,0.000966,0.010922,-0.000527
2015-01-12,-0.007170,0.009434,0.005536,0.014459,0.003636,0.024070,0.006990,0.028302,-0.000737
...,...,...,...,...,...,...,...,...,...
2020-12-25,-0.007970,-0.008330,-0.010097,-0.002983,-0.017151,-0.015650,-0.016002,-0.015741,-0.000786
2020-12-28,-0.009865,-0.004423,-0.008857,-0.006440,0.005949,0.001104,0.007667,0.003457,-0.000866
2020-12-29,0.004530,0.004259,0.006052,0.001342,0.007952,0.005731,0.008774,0.012552,-0.000529
2020-12-30,-0.011198,-0.013840,-0.022395,-0.002960,-0.015741,-0.007498,-0.009048,-0.005176,-0.000636


In [10]:
#时间分段
time_list = w.tdays("2015-01-01", "2020-12-31", "Period=S","Days=Alldays")
time_list.Times

[datetime.date(2015, 6, 30),
 datetime.date(2015, 12, 31),
 datetime.date(2016, 6, 30),
 datetime.date(2016, 12, 31),
 datetime.date(2017, 6, 30),
 datetime.date(2017, 12, 31),
 datetime.date(2018, 6, 30),
 datetime.date(2018, 12, 31),
 datetime.date(2019, 6, 30),
 datetime.date(2019, 12, 31),
 datetime.date(2020, 6, 30),
 datetime.date(2020, 12, 31)]

In [11]:
#定义目标函数：最小化残差平方和
def fun(args):
    x0,x1,x2,x3,x4,x5,x6,y = args
    def sq2(a):
        s = 0
        for j in range(len(y)):
            s += (y[j]-a[0]*x0[j]-a[1]*x1[j]-a[2]*x2[j]-a[3]*x3[j]-a[4]*x4[j]-a[5]*x5[j]-a[6]*x6[j])**2
        return s
    return sq2        

In [12]:
#设置参数约束
def cons(args):
    stock_low = args
    con = (
#            {'type': 'eq', 'fun': lambda a:  a[0]},\
#            {'type': 'eq', 'fun': lambda a:  a[1]},\
#            {'type': 'eq', 'fun': lambda a:  a[2]},\
#            {'type': 'eq', 'fun': lambda a:  a[3]},\
#            {'type': 'eq', 'fun': lambda a:  a[4]},\
#            {'type': 'eq', 'fun': lambda a:  a[5]},\
#            {'type': 'eq', 'fun': lambda a:  a[6]},\
       {'type': 'eq', 'fun': lambda a:  sum(a)-1},\
       {'type': 'ineq', 'fun': lambda a: sum(a[:-1])-stock_low})
    return con

In [24]:
#采用“顺序最小二乘规划方法slsqp”计算各因子的风格系数
slsqp_params = pd.DataFrame(index=time_list.Times,
                            columns=jc_index_code_list+debt_index_code+['R_squared','1-R_squared'])


for i in range(len(time_list.Times)):
    if i == 0:
        raw_price_interval = raw_price[raw_price.index<=time_list.Times[i]]
        price_return_interval = price_return[price_return.index<=time_list.Times[i]]
    else:
        raw_price_interval = raw_price[(raw_price.index<=time_list.Times[i]) & (raw_price.index>=time_list.Times[i-1])]
        price_return_interval = price_return[(price_return.index<=time_list.Times[i]) & (price_return.index>time_list.Times[i-1])]
    x = price_return_interval[jc_index_code_list+debt_index_code]
    y = price_return_interval[fund_code]
    for k in range(len(jc_index_code_list+debt_index_code)):
        vars()['x'+ str(k)] = np.array(x.iloc[:,k])
    y = np.array(y)
    stock_low = 0.8
    args_1 = (x0,x1,x2,x3,x4,x5,x6,y)
    args_2 = stock_low
    bounds = ((0,1), (0,1), (0,1), (0,1), (0,1), (0,1),(0,1))   #各个风格系数的取值范围
    a0 = np.array([0,0,0,0,0,0,0])  #初始化系数的值，注意不同初始化值会影响最后系数的取值大小，但最小目标值差异很小
    vars()['res_'+str(i)] = minimize(fun(args_1), a0, method='SLSQP',constraints=cons(args_2), bounds=bounds)
    sse = vars()['res_'+str(i)].fun
    sst = 0
    for j in range(len(y)):
        sst += (y[j]-y.mean())**2
    R2 = 1-sse/sst
    slsqp_params.loc[time_list.Times[i],jc_index_code_list+debt_index_code] = vars()['res_'+str(i)].x
    slsqp_params.loc[time_list.Times[i],['R_squared','1-R_squared']] = [R2[0],1-R2[0]]

In [25]:
slsqp_params

,399372.SZ,399373.SZ,399374.SZ,399375.SZ,399376.SZ,399377.SZ,H11001.CSI,R_squared,1-R_squared
2015-06-30,0.0806578,0.0767409,0.358764,0,0.387157,2.28767e-17,0.0966808,0.791054,0.208946
2015-12-31,0.313327,0,0,0.382088,5.09575e-17,0.304585,4.98733e-18,0.812624,0.187376
2016-06-30,4.9708e-17,0.200646,0.182224,0.0831933,0.330399,0.00353703,0.2,0.742728,0.257272
2016-12-31,0.126156,0.137563,0.140871,0.1181,0.147983,0.129326,0.2,0.6823,0.3177
2017-06-30,0.182984,0.22735,0.187655,6.76271e-18,0.143037,0.0879577,0.171016,0.516648,0.483352
2017-12-31,0.184377,0.111124,0.388155,0,0.113763,0.00258036,0.2,0.596809,0.403191
2018-06-30,0.142497,0.0903949,0.191626,0.121933,0.187815,0.130416,0.135318,0.824257,0.175743
2018-12-31,0.248754,0.0489836,0.361788,1.37694e-17,0.180845,0,0.15963,0.848024,0.151976
2019-06-30,0.141966,0.163445,0.136738,0.12202,0.126603,0.123934,0.185296,0.871817,0.128183
2019-12-31,0.372293,1.38778e-17,0.298954,0,0.132899,1.38778e-17,0.195854,0.854043,0.145957


In [26]:
x0

array([-0.01973237, -0.01288306, -0.00751162, -0.03775634, -0.02025509,
       -0.00926111, -0.02225047,  0.00412553, -0.03024542,  0.00578514,
        0.00587901,  0.06703292, -0.01842298, -0.01745234, -0.01146244,
       -0.01078399, -0.00815153,  0.05400249, -0.01028853, -0.01671788,
       -0.02521663,  0.00177112, -0.01052302, -0.01264132,  0.00738383,
       -0.0024115 ,  0.01491283,  0.01242644,  0.00150925,  0.00580183,
        0.01582754,  0.00250562, -0.01714658, -0.01364698, -0.00393037,
        0.01864532,  0.01219693, -0.01214141, -0.01728238, -0.00711028,
        0.00889854, -0.00863185, -0.02761439,  0.00302448, -0.00848422,
       -0.00371896,  0.00565883,  0.01545106,  0.03402147, -0.00034571,
        0.0304464 , -0.00704017, -0.01548042, -0.00793489, -0.00914303,
        0.01537404,  0.00893577, -0.01514861,  0.01090337,  0.00601804,
       -0.00961591,  0.01956593, -0.00118552, -0.0050157 , -0.00866662,
       -0.00374955, -0.02530744, -0.0313776 , -0.00568384,  0.00